# 2. Помогаем LLM победить математические задачи

## Импорты

In [164]:
import pandas as pd
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import ToolMessage
from langchain_openai import ChatOpenAI
import math
from tqdm import tqdm
from typing import Any
import os

tqdm.pandas()

In [49]:
math_vac = pd.read_csv("custom_math_tools.csv")

In [50]:
math_vac

,task
0,Периметр прямоугольника со сторонами 1.5 м и 2...
1,Найди сумму площадей круга с радиусом 5 см и к...
2,Переведи числа 20 и 32 из десятичной в двоичну...
3,Переведи числа из двоичной системы счисления 1...
4,Найди объем цилиндра с радиусом 10 см и высото...
5,"Найди длину дуги окружности с радиусом 20 м, о..."
6,Переведи десятичное число 1524 в двоичную сист...
7,"Найти объем куба со стороной 5 см, затем перев..."
8,Стороны треугольника соответственно равны 30 с...
9,Найди объемы двух цилиндров с радиусами 8 см и...


In [ ]:
api_key = os.getenv("OPENROUTER_API")

## Создаем агенты и инструменты для него

In [ ]:
@tool
def add_numbers(num1: float, num2: float) -> float:
    """Складывает два числа и возвращает их сумму."""
    return num1 + num2

@tool
def convert_meters_to_cm(num: float) -> float:
    """Переводит длину из метров в сантиметры."""
    return num * 100

@tool
def convert_cubic_cm_to_liters(num: float) -> float:
    """Переводит объём из кубических сантиметров в литры."""
    return num / 1000

@tool
def compute_rectangle_perimeter(a: float, b: float) -> float:
    """Вычисляет периметр прямоугольника по сторонам a и b."""
    return 2 * (a + b)

@tool
def compute_circle_area(radius: float) -> float:
    """Вычисляет площадь круга по заданному радиусу."""
    return math.pi * radius ** 2

@tool
def compute_cylinder_volume(radius: float, height: float) -> float:
    """Вычисляет объём цилиндра по радиусу основания и высоте."""
    return math.pi * radius ** 2 * height

@tool
def compute_cube_volume(a: float) -> float:
    """Вычисляет объём куба по длине его ребра."""
    return a ** 3

@tool
def convert_binary_to_decimal(binary_number: str) -> int:
    """Переводит число из двоичной системы счисления в десятичную."""
    return int(binary_number, 2)

@tool
def convert_decimal_to_binary(decimal_number: int) -> str:
    """Переводит число из десятичной системы счисления в двоичную (без префикса 0b)."""
    return bin(decimal_number)[2:]

@tool
def get_count_ones(number: str) -> int:
    """Возвращает количество единиц в двоичном представлении числа."""
    return number.count("1")

tools = [
    add_numbers,
    convert_meters_to_cm,
    convert_cubic_cm_to_liters,
    compute_rectangle_perimeter,
    compute_circle_area,
    compute_cylinder_volume,
    compute_cube_volume,
    convert_binary_to_decimal,
    convert_decimal_to_binary,
    get_count_ones,
]

llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    model="xiaomi/mimo-v2-flash:free",
    # model='mistralai/devstral-2512:free',
    # model='openai/gpt-oss-20b:free',
    temperature=0
)


agent = create_agent(model=llm, tools=tools)

## Мучаем модель

In [ ]:
def answer_with_agent(question: str):
    """Отправляет текстовый вопрос в агента и возвращает его ответ."""
    result = agent.invoke({"messages": [("user", question)]})
    return result.get("messages", result)

# Предполагается, что колонка с текстом задачи называется 'task'.
math_vac["agent_full_answer"] = math_vac["task"].progress_apply(answer_with_agent)

math_vac.head()

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:51<00:00,  5.13s/it]


,task,agent_full_answer
0,Периметр прямоугольника со сторонами 1.5 м и 2...,{'messages': [content='Периметр прямоугольника...
1,Найди сумму площадей круга с радиусом 5 см и к...,{'messages': [content='Найди сумму площадей кр...
2,Переведи числа 20 и 32 из десятичной в двоичну...,{'messages': [content='Переведи числа 20 и 32 ...
3,Переведи числа из двоичной системы счисления 1...,{'messages': [content='Переведи числа из двоич...
4,Найди объем цилиндра с радиусом 10 см и высото...,{'messages': [content='Найди объем цилиндра с ...


## Проверяем модель

In [163]:
# Хотел сделать удобный вывод для проверки
def print_agent_answer(task: tuple[Any, pd.Series]):
    for message in task[1]['agent_full_answer']['messages']:
        if isinstance(message, ToolMessage):
            print(f"{message.name} = {message.content}")
        else:
            print(message.content)

In [152]:
iter = math_vac.iterrows()

Верно, внезапный markdown появился, сойдет

In [153]:
print_agent_answer(next(iter))

Периметр прямоугольника со сторонами 1.5 м и 2 м. Отвей дай в сантиметрах.
Для решения задачи сначала вычислим периметр прямоугольника, а затем переведем его из метров в сантиметры.
compute_rectangle_perimeter = 7.0
Теперь переведем 7 метров в сантиметры:
convert_meters_to_cm = 700.0
**Ответ:** Периметр прямоугольника со сторонами 1.5 м и 2 м равен **700 см**.


Тоже верно, внезапный latex появился

In [154]:
print_agent_answer(next(iter))

Найди сумму площадей круга с радиусом 5 см и круга с радиусом 3 см.
Для решения задачи нужно вычислить площади двух кругов и найти их сумму. Используем формулу площади круга $S = \pi r^2$.

1. Площадь первого круга с радиусом 5 см: $S_1 = \pi \cdot 5^2 = 25\pi$.
2. Площадь второго круга с радиусом 3 см: $S_2 = \pi \cdot 3^2 = 9\pi$.
3. Сумма площадей: $S_{\text{общ}} = 25\pi + 9\pi = 34\pi$.

Для точного вычисления используем инструмент `compute_circle_area` для каждого круга, а затем сложим результаты.
compute_circle_area = 78.53981633974483
compute_circle_area = 28.274333882308138
add_numbers = 106.81415022205297
Сумма площадей кругов с радиусами 5 см и 3 см составляет приблизительно **106.81 квадратных сантиметров** (или точно $34\pi$ см²).


Тоже верно, ток теперь ответ подробный

In [155]:
print_agent_answer(next(iter))

Переведи числа 20 и 32 из десятичной в двоичную систему счисления и найди сумму единиц.
Сначала я переведу числа 20 и 32 из десятичной системы счисления в двоичную, а затем посчитаю количество единиц в полученных двоичных числах и найду их сумму.
convert_decimal_to_binary = 10100
convert_decimal_to_binary = 100000
Теперь, когда у нас есть двоичные представления чисел:
- 20 в двоичной системе: 10100
- 32 в двоичной системе: 100000

Теперь я посчитаю количество единиц в каждом из этих чисел.
get_count_ones = 2
get_count_ones = 1
Теперь я сложу количество единиц в двоичных представлениях чисел 20 и 32.
add_numbers = 3.0
**Ответ:**
- Двоичное представление числа 20: 10100 (2 единицы)
- Двоичное представление числа 32: 100000 (1 единица)
- Сумма единиц: 3


Верно, без latex решил подпись системы счиления сделать

In [156]:
print_agent_answer(next(iter))

Переведи числа из двоичной системы счисления 1101011010 и 101011 и посчитай их сумму.
Сначала я переведу оба числа из двоичной системы в десятичную, а затем сложу их.
convert_binary_to_decimal = 858
convert_binary_to_decimal = 43
Теперь сложу полученные десятичные числа: 858 и 43.
add_numbers = 901.0
**Ответ:**  
Переведённые числа: 1101011010₂ = 858₁₀, 101011₂ = 43₁₀.  
Их сумма: 858 + 43 = **901**.


Верно, в этото раз без явного ответа

In [157]:
print_agent_answer(next(iter))

Найди объем цилиндра с радиусом 10 см и высотой 30 см, переведи результат в литры.
Сначала я вычислю объём цилиндра, а затем переведу его из кубических сантиметров в литры.
compute_cylinder_volume = 9424.77796076938
Теперь переведу объём из кубических сантиметров в литры.
convert_cubic_cm_to_liters = 9.42477796076938
Объём цилиндра с радиусом 10 см и высотой 30 см составляет примерно **9,425 литра**.


В этот раз инструмента не было, и даже без галюцинаций, ого, xiaomi топ. Латех сломался, а так верно

In [158]:
print_agent_answer(next(iter))

Найди длину дуги окружности с радиусом 20 м, образованной центральным углом 60°, затем переведи результат в сантиметры.
Для решения задачи выполним следующие шаги:
1. **Вычислим длину дуги** по формуле:  
   \( L = 2\pi R \cdot \frac{\alpha}{360^\circ} \),  
   где \( R = 20 \) м, \( \alpha = 60^\circ \).  
   Подставляем значения:  
   \( L = 2\pi \cdot 20 \cdot \frac{60}{360} = \frac{20\pi}{3} \) м.

2. **Переведём метры в сантиметры**, используя коэффициент \( 1 \) м = \( 100 \) см:  
   \( L_{\text{см}} = \frac{20\pi}{3} \cdot 100 = \frac{2000\pi}{3} \) см.

Для точного вычисления воспользуемся инструментом `convert_meters_to_cm`, передав в него значение длины дуги в метрах (округлим до 2 знаков после запятой для удобства):  
\( L \approx 20,94 \) м → \( 2094 \) см.

Итоговый ответ:  
**Длина дуги ≈ 2094 см** (или \( \frac{2000\pi}{3} \) см).
convert_meters_to_cm = 2094.0
**Длина дуги окружности** с радиусом 20 м и центральным углом 60° составляет **2094 см** (точнее: \( \frac{2000

Опять забил на ответ, но в целом верно

In [159]:
print_agent_answer(next(iter))

Переведи десятичное число 1524 в двоичную систему счисления и найди количество единиц в его двоичном представлении.
Сначала я переведу число 1524 в двоичную систему счисления, а затем посчитаю количество единиц в полученном двоичном числе.
convert_decimal_to_binary = 10111110100
Теперь, когда у меня есть двоичное представление числа 1524 — 10111110100, я посчитаю количество единиц в этом числе.
get_count_ones = 7
Десятичное число 1524 в двоичной системе счисления равно 10111110100. В этом двоичном числе содержится 7 единиц.


Верно

In [160]:
print_agent_answer(next(iter))

Найти объем куба со стороной 5 см, затем перевести результат в двоичную систему счисления.
Сначала я найду объем куба со стороной 5 см, а затем переведу результат в двоичную систему счисления.
compute_cube_volume = 125.0
Теперь переведу 125 в двоичную систему:
convert_decimal_to_binary = 1111101
Объем куба со стороной 5 см равен 125 см³. В двоичной системе счисления это число записывается как 1111101.


Ого ответ появился, верно

In [161]:
print_agent_answer(next(iter))

Стороны треугольника соответственно равны 30 см, 40 см и 110010 см (переведи это число из двоичной системы счисления в десятичную). Найди площадь этого треугольника.
Сначала нужно перевести число из двоичной системы счисления в десятичную. Для этого воспользуемся функцией `convert_binary_to_decimal`.
convert_binary_to_decimal = 50
Теперь у нас есть все стороны треугольника: 30 см, 40 см и 50 см. Заметим, что это прямоугольный треугольник, так как 30² + 40² = 900 + 1600 = 2500 = 50². Площадь прямоугольного треугольника равна половине произведения катетов: (30 * 40) / 2 = 600 см². Но для проверки можно использовать формулу Герона.

Вычислим полупериметр: (30 + 40 + 50) / 2 = 60 см. Площадь по формуле Герона: √[60*(60-30)*(60-40)*(60-50)] = √[60*30*20*10] = √36000 = 600 см².

Ответ: 600 см².


Ого, прям очень подробно рассписано, в целом верно

In [162]:
print_agent_answer(next(iter))

Найди объемы двух цилиндров с радиусами 8 см и 12 см и высотами 20 см и 25 см соответственно, затем сложи эти объемы и переведи сумму в литры.
Сначала я вычислю объёмы двух цилиндров, затем сложу их и переведу в литры. Для этого я воспользуюсь формулой объёма цилиндра: V = πr²h. Поскольку в моих инструментах есть функция `compute_cylinder_volume`, я буду использовать её. Также мне понадобится функция `convert_cubic_cm_to_liters` для перевода кубических сантиметров в литры. Наконец, чтобы сложить объёмы, я воспользуюсь функцией `add_numbers`.

Итак, шаги:
1. Вычислить объём первого цилиндра с радиусом 8 см и высотой 20 см.
2. Вычислить объём второго цилиндра с радиусом 12 см и высотой 25 см.
3. Сложить два объёма.
4. Перевести сумму из кубических сантиметров в литры.

Начну с первого цилиндра.
compute_cylinder_volume = 4021.238596594935
Теперь вычислю объём второго цилиндра.
compute_cylinder_volume = 11309.733552923255
Теперь сложу два объёма. Первый объём — 4021.238596594935, второй — 